In [176]:
import pandas as pd

In [215]:
df = pd.read_csv('nba_player_data_2015.csv')

In [217]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [218]:
df.head()

,+/1,3P,3PA,AST,BLK,Date,FG,FGA,FT,FTA,...,TOV,TRB,Team,minutes,name,opp_score,started,team_score,is_Home,Team_Win
0,23.0,0.0,0.0,3.0,2.0,10/27/2015,6.0,16.0,6.0,10.0,...,2.0,19.0,Detroit,37:09,Andre Drummond,94,1,106,0,1
1,17.0,1.0,4.0,4.0,0.0,10/27/2015,6.0,19.0,5.0,6.0,...,0.0,10.0,Detroit,37:05,Marcus Morris,94,1,106,0,1
2,17.0,4.0,7.0,1.0,0.0,10/27/2015,7.0,14.0,3.0,3.0,...,2.0,4.0,Detroit,37:03,Kentavious Caldwell-Pope,94,1,106,0,1
3,20.0,3.0,6.0,3.0,1.0,10/27/2015,6.0,12.0,1.0,2.0,...,3.0,7.0,Detroit,34:26,Ersan Ilyasova,94,1,106,0,1
4,26.0,2.0,4.0,5.0,0.0,10/27/2015,4.0,10.0,5.0,5.0,...,2.0,8.0,Detroit,32:07,Reggie Jackson,94,1,106,0,1


In [219]:
df['spread'] = df['team_score']-df['opp_score']

In [220]:
groups= df.groupby(['Game_Id','Team'],sort=False).sum()

In [221]:
matrix = groups.as_matrix()

In [222]:
ids = []
for game in df['Game_Id'].unique():
    ids.append(game+ '%aa'+game.split('_')[1])
    ids.append(game+ '%aa'+game.split('_')[2])


In [223]:
new_frame = pd.DataFrame(matrix, columns = groups.columns, index = ids)

In [224]:
team_data = new_frame[['3P','3PA', 'AST', 'BLK', 'FG', 'FGA', 'FT', 'FTA', 'ORB',
       'PF', 'PTS', 'STL', 'TOV', 'TRB','is_Home','Team_Win']]

In [225]:
team_data = team_data.reset_index()

In [226]:
team_data['Team'] = team_data['index'].apply(lambda x: x.split('%aa')[1])

In [227]:
team_data['index'] = team_data['index'].apply(lambda x: x.split('%aa')[0])

In [228]:
team_data['is_Home'] = team_data['is_Home'].apply(lambda x: x>0).astype(int)
team_data['Team_Win'] = team_data['Team_Win'].apply(lambda x: x>0).astype(int)

In [229]:
team_data.head()

,index,3P,3PA,AST,BLK,FG,FGA,FT,FTA,ORB,PF,PTS,STL,TOV,TRB,is_Home,Team_Win,Team
0,10/27/2015_Atlanta_Detroit,12.0,29.0,23.0,3.0,37.0,96.0,20.0,26.0,23.0,15.0,106.0,5.0,15.0,59.0,0,1,Atlanta
1,10/27/2015_Atlanta_Detroit,8.0,27.0,22.0,4.0,37.0,82.0,12.0,15.0,7.0,25.0,94.0,9.0,15.0,40.0,1,0,Detroit
2,10/27/2015_Chicago_Cleveland,9.0,29.0,26.0,7.0,38.0,94.0,10.0,17.0,11.0,21.0,95.0,5.0,10.0,50.0,0,0,Chicago
3,10/27/2015_Chicago_Cleveland,7.0,19.0,13.0,10.0,37.0,87.0,16.0,23.0,7.0,22.0,97.0,6.0,13.0,47.0,1,1,Cleveland
4,10/27/2015_Golden State_New Orleans,6.0,18.0,21.0,3.0,35.0,83.0,19.0,27.0,8.0,26.0,95.0,9.0,18.0,33.0,0,0,Golden State


In [230]:
both_teams_total = team_data.groupby('index',sort=False).sum().reset_index()
##doing weird things. 
weird = pd.merge(both_teams_total,team_data,on='index')

In [231]:
#fill in opponent data in an insufficient way
team_data['opp_3P'] = weird['3P_x']-team_data['3P']
team_data['opp_3PA'] = weird['3PA_x']-team_data['3PA']
team_data['opp_AST'] = weird['AST_x']-team_data['AST']
team_data['opp_BLK'] = weird['BLK_x']-team_data['BLK']
team_data['opp_FG'] = weird['FG_x']-team_data['FG']
team_data['opp_FGA'] = weird['FGA_x']-team_data['FGA']
team_data['opp_FT'] = weird['FT_x']-team_data['FT']
team_data['opp_FTA'] = weird['FTA_x']-team_data['FTA']
team_data['opp_PTS'] = weird['PTS_x']-team_data['PTS']
team_data['opp_ORB'] = weird['ORB_x']-team_data['ORB']
team_data['opp_STL'] = weird['STL_x']-team_data['STL']
team_data['opp_PF'] = weird['PF_x']-team_data['PF']
team_data['opp_TOV'] = weird['TOV_x']-team_data['TOV']

In [232]:
def opponent(row):
    if row['Team'] == row['index'].split('_')[1]:
        return row['index'].split('_')[2]
    else:
        return row['index'].split('_')[1]

In [233]:
team_data['opponent'] =team_data.apply(opponent, axis=1)

In [234]:
team_data['Game_Id'] = team_data['index']
team_data.drop('index',axis=1, inplace=True)

In [237]:
team_data['Game_Id']

0                 10/27/2015_Atlanta_Detroit
1                 10/27/2015_Atlanta_Detroit
2               10/27/2015_Chicago_Cleveland
3               10/27/2015_Chicago_Cleveland
4        10/27/2015_Golden State_New Orleans
5        10/27/2015_Golden State_New Orleans
6             10/28/2015_Boston_Philadelphia
7             10/28/2015_Boston_Philadelphia
8                10/28/2015_Brooklyn_Chicago
9                10/28/2015_Brooklyn_Chicago
10                   10/28/2015_Detroit_Utah
11                   10/28/2015_Detroit_Utah
12                 10/28/2015_Houston_Denver
13                 10/28/2015_Houston_Denver
14            10/28/2015_LA Lakers_Minnesota
15            10/28/2015_LA Lakers_Minnesota
16              10/28/2015_Memphis_Cleveland
17              10/28/2015_Memphis_Cleveland
18                10/28/2015_Miami_Charlotte
19                10/28/2015_Miami_Charlotte
20             10/28/2015_Milwaukee_New York
21             10/28/2015_Milwaukee_New York
22      10

In [253]:
##get rid of games that are playoffs
team_data = team_data.iloc[:2460]
team_data.to_csv('team_boxscores_2015.csv')

In [258]:
df = pd.read_csv('nba_player_data_2015.csv')
df = df.iloc[:31430]
df.to_csv('correct_nba_player_data_2015.csv')

In [257]:
df.shape

(32548, 27)